In [1]:
!pip install --quiet opencv-python ultralytics
!pip install --quiet iterative-stratification

# Detecting Group Emotions

some text








## Overview

description of the problem

## Data

Comment about problems with the data.

Load the data.

In [10]:
import pandas as pd
import ast
import os
from pathlib import Path
from google.colab import drive
from utilsJ import *
import cv2
from ultralytics import YOLO
import torch

drive.mount("/content/drive")
%cd 'drive/Othercomputers/My Computer (1)/EmotionTeller-github'

from utilsJ import *

data_root       = Path('Data')                # Folder where all data sources are stored
dataset_path    = data_root/'ImageData'       # List of folders containing images in .jpg format
data_meta       = ['emotic-relabelled.csv',
                'hgel-relabelled.csv']        # List of metadata corresponding to previous list of folders in .csv format. In our case we relabelled some of the data, so this is different from original metadata.
meta_root       = Path('Metadata')            # Folder where the previous .csv files are located
yolo_dir        = Path('YOLO_training')       # Folder where the results and data splitting will take place for YOLO

emo_dic = {'Neutral':0,'Happy':1,'Surprise':2,'Sad':3,'Angry':4,'Fear':5,'Disgust':6}

os.makedirs(yolo_dir, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/Othercomputers/My Computer (1)/EmotionTeller-github'
/content/drive/Othercomputers/My Computer (1)/EmotionTeller-github


In [11]:
df_train = pd.read_csv(meta_root/'train_meta.csv')
df_train['objects'] = df_train['objects'].apply(lambda x: ast.literal_eval(x))

# Check if the file exists for each row
df_train['file_exists'] = df_train.apply(lambda x: os.path.exists(dataset_path/x.file_name),axis = 1)

# Drop rows where the file does not exist
df_train = df_train[df_train.file_exists]
df_train.drop(columns=['file_exists'], inplace=True)

df_test = pd.read_csv(meta_root/'test_meta.csv')
df_test['objects'] = df_test['objects'].apply(lambda x: ast.literal_eval(x))

# Check if the file exists for each row
df_test['file_exists'] = df_test.apply(lambda x: os.path.exists(dataset_path/x.file_name),axis = 1)

# Drop rows where the file does not exist
df_test = df_test[df_test.file_exists]
df_test.drop(columns=['file_exists'], inplace=True)

# Add the path to the image
df_train['path_to_img'] = df_train.apply(lambda x: dataset_path/x.file_name,axis=1)
df_test['path_to_img'] = df_test.apply(lambda x: dataset_path/x.file_name,axis=1)

## Models



### Model 1 - YOLO

Mention in which notebook training was done and any other details: `YOLOfinetune.ipynb`.

Load results, show at least one example and metrics.

### Model 2 - Two-step model

Mention in which notebook training was done and any other details: `YOLOfinetune.ipynb`.

Load results, show at least one example and metrics (might have to specify the extra metrics one can consider here).

### Evaluation and Results

Below we compute the metrics for the `yolov11m` model that we finetuned. Due to the nature of our predictions, we use an IoU threshold of 0.05. A bigger dataset would yield a more robust model, this would allow us to increase this threshold.

In [12]:
weights_path = Path('YOLO_training/runs')

name = 'yolo11m_finetuned4'
medium_finetuned = YOLO(weights_path / name /'weights'/'best.pt')

train_preds = medium_finetuned.predict(list(df_train['path_to_img']),verbose=False)
train_combined = combine_gt_pred(df_train,train_preds,emo_dic)
del train_preds
torch.cuda.empty_cache() # Important to not run out of GPU memory

test_preds = medium_finetuned.predict(list(df_test['path_to_img']),verbose=False)
test_combined = combine_gt_pred(df_test,test_preds,emo_dic)
del test_preds
torch.cuda.empty_cache() # Important to not run out of GPU memory

In [5]:
detect_metrics(train_combined, test_combined)

Train Metrics:
Precision: 0.7028
Recall: 0.6652
mAP50: 0.4333
mAP50-95: 0.0000

Test Metrics:
Precision: 0.5419
Recall: 0.3702
mAP50: 0.1341
mAP50-95: 0.0000


The cell below allows to visualize the predictions of our model.

In [9]:
drop_display(train_combined,test_combined)

Train Set:


Dropdown(description='Select Train Index:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,…

Output()


Test Set:


Dropdown(description='Select Test Index:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, …

Output()

We now compute the metrics for the `yolov11m-face` model that we finetuned.

In [6]:
name = 'yolo11m_face_finetuned2'
medium_face_finetuned = YOLO(weights_path / name /'weights'/'best.pt')

train_face_preds = medium_face_finetuned.predict(list(df_train['path_to_img']),verbose=False, iou=0.5)
train_face_combined = combine_gt_pred(df_train,train_face_preds,emo_dic)
del train_face_preds
torch.cuda.empty_cache() # Important to not run out of GPU memory

test_face_preds = medium_face_finetuned.predict(list(df_test['path_to_img']),verbose=False, iou=0.5)
test_face_combined = combine_gt_pred(df_test,test_face_preds,emo_dic)
del test_face_preds
torch.cuda.empty_cache() # Important to not run out of GPU memory

In [7]:
detect_metrics(train_face_combined, test_face_combined)

Train Metrics:
Precision: 0.5399
Recall: 0.2396
mAP50: 0.1284
mAP50-95: 0.0000

Test Metrics:
Precision: 0.5000
Recall: 0.2176
mAP50: 0.1957
mAP50-95: 0.0000


The cell below allows to visualize the predictions of our model.

In [13]:
drop_display(train_face_combined,test_face_combined)

Train Set:


Dropdown(description='Select Train Index:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,…

Output()


Test Set:


Dropdown(description='Select Test Index:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, …

Output()

### App

## Further improvements